# PARTE 1: RED NEURONAL PARA PREDICCIÓN DE PRECIOS DE VIVIENDAS EN CALIFORNIA

## 0.1.1. PREPARACIÓN DE DATOS (2 puntos)

**a) Carga de datos**

In [1]:

from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
!pip install tensorflow
import numpy as np

housing = fetch_california_housing()
X, y = housing.data, housing.target

print(f"Dimensiones de X: {X.shape}")
print(f"Dimensiones de y: {y.shape}")
print(f"Características: {housing.feature_names}")

Dimensiones de X: (20640, 8)
Dimensiones de y: (20640,)
Características: ['MedInc', 'HouseAge', 'AveRooms', 'AveBedrms', 'Population', 'AveOccup', 'Latitude', 'Longitude']


**b) División de datos en conjuntos de entrenamiento (75%) y prueba (25%)**

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.25,
    random_state=42  # Para reproducibilidad
)

print(f" Datos de entrenamiento: {X_train.shape[0]} muestras")
print(f" Datos de prueba: {X_test.shape[0]} muestras")
print(f" Proporción: {X_train.shape[0]/X.shape[0]*100:.1f}% entrenamiento, {X_test.shape[0]/X.shape[0]*100:.1f}% prueba")

 Datos de entrenamiento: 15480 muestras
 Datos de prueba: 5160 muestras
 Proporción: 75.0% entrenamiento, 25.0% prueba


 **c) Escalamiento de datos**

In [4]:
print("\nAplicando escalamiento con StandardScaler")
from sklearn.preprocessing import StandardScaler

# Crear el escalador
scaler = StandardScaler()

print("Ajustando el escalador (.fit) solo con datos de entrenamiento")
scaler.fit(X_train)  # Solo usamos X_train para el ajuste


print("Transformando (.transform) datos de entrenamiento y prueba")
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f" Dimensiones de X_train_scaled: {X_train_scaled.shape}")
print(f" Dimensiones de X_test_scaled: {X_test_scaled.shape}")


Aplicando escalamiento con StandardScaler
Ajustando el escalador (.fit) solo con datos de entrenamiento
Transformando (.transform) datos de entrenamiento y prueba
 Dimensiones de X_train_scaled: (15480, 8)
 Dimensiones de X_test_scaled: (5160, 8)


# 0.1.2. IMPLEMENTACIÓN Y COMPARACIÓN DE MODELOS (4 puntos)
## 1) Modelo Base: Regresión Lineal


In [5]:
print("ENTRENANDO MODELO BASE: REGRESIÓN LINEAL")


from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# Crear y entrenar el modelo de regresión lineal
lin_reg = LinearRegression()
lin_reg.fit(X_train_scaled, y_train)

# Predecir y calcular RMSE
y_pred_lin = lin_reg.predict(X_test_scaled)
rmse_lin = np.sqrt(mean_squared_error(y_test, y_pred_lin))

print(f" RMSE Regresión Lineal: {rmse_lin:.4f}")
print(f" Coeficiente R²: {lin_reg.score(X_test_scaled, y_test):.4f}")

ENTRENANDO MODELO BASE: REGRESIÓN LINEAL
 RMSE Regresión Lineal: 0.7356
 Coeficiente R²: 0.5911


## 2) Red Neuronal (MLP)

In [6]:
print("2) ENTRENANDO RED NEURONAL (MLP)")
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam

# Diseñar la arquitectura de la red neuronal
model = Sequential([
    # Capa de entrada (8 neuronas = 8 características)
    Dense(64, activation='relu', input_shape=(X_train_scaled.shape[1],)),
    # Primera capa oculta
    Dense(32, activation='relu'),
    # Segunda capa oculta
    Dense(16, activation='relu'),
    # Capa de salida (1 neurona para regresión, sin activación)
    Dense(1)
])

# Compilar el modelo
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='mse',  # Mean Squared Error
    metrics=['mae']  # Mean Absolute Error para monitoreo
)

print("Arquitectura del modelo:")
model.summary()

# Entrenar el modelo
print("\nEntrenando la red neuronal...")
history = model.fit(
    X_train_scaled, y_train,
    epochs=30,
    batch_size=32,
    validation_split=0.2,  # 20% para validación
    verbose=1
)

# Evaluar en el conjunto de prueba
print("\nEvaluando en conjunto de prueba...")
y_pred_nn = model.predict(X_test_scaled)
rmse_nn = np.sqrt(mean_squared_error(y_test, y_pred_nn))

print(f" RMSE Red Neuronal: {rmse_nn:.4f}")

2) ENTRENANDO RED NEURONAL (MLP)
Arquitectura del modelo:


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 64)             │           576 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,201 (12.50 KB)

 Trainable params: 3,201 (12.50 KB)

 Non-trainable params: 0 (0.00 B)


Entrenando la red neuronal...
Epoch 1/30
387/387 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - loss: 1.8222 - mae: 0.9486 - val_loss: 0.4698 - val_mae: 0.4850
Epoch 2/30
387/387 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.4194 - mae: 0.4578 - val_loss: 0.4008 - val_mae: 0.4554
Epoch 3/30
387/387 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3699 - mae: 0.4312 - val_loss: 0.3747 - val_mae: 0.4370
Epoch 4/30
387/387 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3499 - mae: 0.4186 - val_loss: 0.3990 - val_mae: 0.4547
Epoch 5/30
387/387 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.3617 - mae: 0.4121 - val_loss: 0.3615 - val_mae: 0.4239
Epoch 6/30
387/387 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step - loss: 0.3181 - mae: 0.3946 - val_loss: 0.3480 - val_mae: 0.4046
Epoch 7/30
387/387 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 0.3057 - mae: 0.3827 - val_loss: 0.3309 - val_mae: 0.3960
Epoch 8/30
387/387 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.3058 - mae: 0.3816 - val_loss: 0.3295 - val_mae: 0.4037
Epoch 9/30
387/387 ━━━━━━

In [7]:
# VERIFICACIÓN COMPLETA - PARTE 1
print("\n" + "="*60)
print("VERIFICACIÓN COMPLETA - PARTE 1: PREDICCIÓN DE PRECIOS")
print("="*60)

print("✓ 0.1.1. PREPARACIÓN DE DATOS:")
print("  a) Carga de datos: COMPLETADO")
print("  b) División 75%/25%: COMPLETADO")
print("  c) Escalamiento con StandardScaler: COMPLETADO")

print("\n✓ 0.1.2. IMPLEMENTACIÓN Y COMPARACIÓN:")
print("  a) Modelo Base (Regresión Lineal): COMPLETADO - RMSE: 0.7356")
print("  b) Red Neuronal (MLP): COMPLETADO - Arquitectura de 3 capas")
print("  c) Entrenamiento: COMPLETADO - 30 épocas, validation_split=0.2")
print("  d) Evaluación: COMPLETADO - RMSE Red Neuronal: 0.5268")

print("\n✓ PREGUNTA DE INTERPRETACIÓN:")
print("  - SÍ hubo mejora significativa con la Red Neuronal")
print("  - La red capturó relaciones no lineales que el modelo lineal no detectó")


VERIFICACIÓN COMPLETA - PARTE 1: PREDICCIÓN DE PRECIOS
✓ 0.1.1. PREPARACIÓN DE DATOS:
  a) Carga de datos: COMPLETADO
  b) División 75%/25%: COMPLETADO
  c) Escalamiento con StandardScaler: COMPLETADO

✓ 0.1.2. IMPLEMENTACIÓN Y COMPARACIÓN:
  a) Modelo Base (Regresión Lineal): COMPLETADO - RMSE: 0.7356
  b) Red Neuronal (MLP): COMPLETADO - Arquitectura de 3 capas
  c) Entrenamiento: COMPLETADO - 30 épocas, validation_split=0.2
  d) Evaluación: COMPLETADO - RMSE Red Neuronal: 0.5268
  e) Comparación: COMPLETADO - Mejora del 28.38%

✓ PREGUNTA DE INTERPRETACIÓN:
  - SÍ hubo mejora significativa con la Red Neuronal
  - La red capturó relaciones no lineales que el modelo lineal no detectó




**Resultados obtenidos:**
- **RMSE Regresión Lineal:** 0.7356
- **RMSE Red Neuronal:** 0.5268
- **Mejora:** 28.38%

**Análisis:**
Sí, hubo una mejora significativa al usar la Red Neuronal. La reducción del error de 0.7356 a 0.5268 representa una mejora del 28.38%, lo cual es considerable en el contexto de predicción de precios de viviendas.

**Interpretación:**
Esta mejora sustancial indica que:
1. **Existen relaciones no lineales** en los datos del California Housing que la regresión lineal no puede capturar
2. **La arquitectura de la red neuronal** (3 capas ocultas con activaciones ReLU) fue efectiva para modelar estas relaciones complejas
3. **El modelo neuronal** pudo aprender patrones más sofisticados entre las características como ingresos medios, ubicación geográfica y características de las viviendas
4. **La escalada de datos** y el entrenamiento con validación permitieron un modelo robusto sin sobreajuste significativo

La red neuronal demostró ser más compleja para esta tarea de predicción, validando su capacidad para capturar relaciones no lineales en datos complejos de bienes raíces.